In [1]:
colab = False

if colab == True:
    !pip install nnunetv2
    !pip install torchio
    

In [2]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
import shutil
from collections import OrderedDict
import json
import numpy as np
import nnunetv2

if colab:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)
    base_dir = '/content/drive/My Drive/Colab Notebooks/tumor-segmentation'
    os.chdir(base_dir)
    torch.cuda.empty_cache()

    !git clone https://github.com/MIC-DKFZ/nnUNet.git
    !git clone https://github.com/NVIDIA/apex

    training_size = 400


else:
    base_dir = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'
    training_size = 200

from utils import validate_segmentation, plot_prediction, dice_score
import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example

path_to_mask = 'data/patients/labels/'
path_to_imgs = 'data/patients/imgs/'
path_to_controls = 'data/controls/imgs/' #healthy individuals (no tumors)
dataset_path = 'data.zip'
dataset_dir_name = 'data'
dataset_dir = Path(dataset_dir_name)

if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url}
    !unzip -qq {dataset_path}

In [3]:
images_dir = dataset_dir / 'patients/imgs'
labels_dir = dataset_dir / 'patients/labels'
controls_dir = dataset_dir / 'controls/imgs'
image_paths = sorted(images_dir.glob('*.png'))
label_paths = sorted(labels_dir.glob('*.png'))
control_paths = sorted(controls_dir.glob('*.png'))

dataset_ID = 1

# dataset = torchio_compose_train(image_paths, label_paths, control_paths, 
#                                 cropsize=(400,991), train_size = training_size,
#                                 dataset_ID=dataset_ID)

In [4]:
# Define the base path for your nnUNet directories
base_path = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'

# Set the environment variables
os.environ['nnUNet_raw'] = os.path.join(base_path, 'nnUNet_raw')
os.environ['nnUNet_preprocessed'] = os.path.join(base_path, 'nnUNet_preprocessed')
os.environ['nnUNet_results'] = os.path.join(base_path, 'nnUNet_results')

In [5]:
!nnUNetv2_plan_and_preprocess -h

usage: nnUNetv2_plan_and_preprocess [-h] [-d D [D ...]] [-fpe FPE]
                                    [-npfp NPFP] [--verify_dataset_integrity]
                                    [--no_pp] [--clean] [-pl PL]
                                    [-gpu_memory_target GPU_MEMORY_TARGET]
                                    [-preprocessor_name PREPROCESSOR_NAME]
                                    [-overwrite_target_spacing OVERWRITE_TARGET_SPACING [OVERWRITE_TARGET_SPACING ...]]
                                    [-overwrite_plans_name OVERWRITE_PLANS_NAME]
                                    [-c C [C ...]] [-np NP [NP ...]]
                                    [--verbose]

options:
  -h, --help            show this help message and exit
  -d D [D ...]          [REQUIRED] List of dataset IDs. Example: 2 4 5. This
                        will run fingerprint extraction, experiment planning
                        and preprocessing for these datasets. Can of course
                        al

In [6]:
!nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity

Fingerprint extraction...
Dataset001
Using <class 'nnunetv2.imageio.natural_image_reager_writer.NaturalImage2DIO'> as reader/writer
Error: Unexpected labels found in file /home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation/nnUNet_raw/Dataset001/labelsTr/patient_0000.png.
Expected: [0, 1]
Found: [  0. 255.]
Error: Unexpected labels found in file /home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation/nnUNet_raw/Dataset001/labelsTr/patient_0001.png.
Expected: [0, 1]
Found: [  0. 255.]
Error: Unexpected labels found in file /home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation/nnUNet_raw/Dataset001/labelsTr/patient_0002.png.
Expected: [0, 1]
Found: [  0. 255.]
Error: Unexpected labels found in file /home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation/nnUNet_raw/Dataset001/labelsTr/patient_0003.png.
Expected: [0, 1]
Found: [  0. 255.]
Error: Unexpected labels found in file /home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation/nnUNet_raw/Dataset001/labelsTr/patient_0004.png.
Expe

In [7]:
!nnUNetv2_train -h

usage: nnUNetv2_train [-h] [-tr TR] [-p P]
                      [-pretrained_weights PRETRAINED_WEIGHTS]
                      [-num_gpus NUM_GPUS] [--use_compressed] [--npz] [--c]
                      [--val] [--val_best] [--disable_checkpointing]
                      [-device DEVICE]
                      dataset_name_or_id configuration fold

positional arguments:
  dataset_name_or_id    Dataset name or ID to train with
  configuration         Configuration that should be trained
  fold                  Fold of the 5-fold cross-validation. Should be an int
                        between 0 and 4.

options:
  -h, --help            show this help message and exit
  -tr TR                [OPTIONAL] Use this flag to specify a custom trainer.
                        Default: nnUNetTrainer
  -p P                  [OPTIONAL] Use this flag to specify a custom plans
                        identifier. Default: nnUNetPlans
  -pretrained_weights PRETRAINED_WEIGHTS
                        [O

## The following only works with CUDA

In [8]:
# !nnUNetv2_train Dataset001 2d 4 -device 'cpu'